In [1]:
import pandas as pd
import sqlite3

In [37]:
conn = sqlite3.connect('../data/checking-logs.sqlite.sqlite_copy')

In [4]:
pd.io.sql.read_sql('pragma table_info(test)', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


In [5]:
pd.io.sql.read_sql('select * from test limit 10', conn)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


## Минимальная разница между первым коммитом и делайном

In [13]:
df_min = pd.io.sql.read_sql(
    """
    select uid, MIN(diff)
    from
        (select uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as diff
        from test t
        left join deadlines dl on t.labname = dl.labs
        where not t.labname = 'project1'
        order by 2)gg
    limit 1
    """
    , conn)
df_min

,uid,MIN(diff)
0,user_30,-202


## Максимальная

In [14]:
df_max = pd.io.sql.read_sql(
    """
    select uid, MAX(diff)
    from
        (select uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as diff
        from test t
        left join deadlines dl on t.labname = dl.labs
        where not t.labname = 'project1'
        order by 2)gg
    limit 1
    """
    , conn)
df_max

,uid,MAX(diff)
0,user_25,-2


## Среднее значение

In [15]:
df_avg = pd.io.sql.read_sql(
    """
    select AVG(diff)
    from
        (select uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as diff
        from test t
        left join deadlines dl on t.labname = dl.labs
        where not t.labname = 'project1'
        order by 2)gg
    limit 1
    """
    , conn)
df_avg

,AVG(diff)
0,-89.125


## Вечесляем коэффициент корреляции

In [40]:

views_diff = pd.io.sql.read_sql(
    """
    select ll.uid, ll.avg_diff, vs.pageviews
    from
    (select uid, avg(avg_diff) as avg_diff from
        (select uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 as integer) as avg_diff
        from test t
        left join deadlines dl on t.labname = dl.labs
        where not t.labname = 'project1'
        group by uid
        ) kk 
    group by uid) ll
    left join
        (select uid, count(*) as pageviews
        from  pageviews
        group by uid
        ) vs 
        on ll.uid = vs.uid
    """,
    conn
)
print(views_diff)
views_diff.corr()

        uid  avg_diff  pageviews
0    user_1      -6.0         28
1   user_10     -39.0         89
2   user_14    -200.0        143
3   user_17     -81.0         47
4   user_18      -4.0          3
5   user_19    -148.0         16
6   user_21    -126.0         10
7   user_25    -148.0        179
8   user_28     -98.0        149
9    user_3     -75.0        317
10  user_30    -202.0          3


,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


In [41]:
conn.close()